# CH03 General Traits of Good Code

#### 목표
- 견고한 소프트웨어(SW)의 개념을 이해
- 작업 중 잘못된 데이터를 다루는 법
- 새로운 요구 사항을 쉽게 받아들이고 확장할 수 있는 유지보구사 쉬운 SW 설계
- 재사용 가능한 SW 설계
- 개발팀의 생산성을 높이는 효율적인 코드 작성 

#### 계약에 의한 디자인
계약에 의한 디자인이란, 관계자가 기대하는 바를 암묵적으로 코드에 삽입하는 대신 양측이 동의하는 계약을 먼저 한 다음, 계약을 어겼을 경우는 명시적으로 왜 계속할 수 없는지 예외를 발생시키는 것을 의미하며 구체적으로는 SW 컴포넌트간의 통신 중에 반드시 지켜져야 할 몇 가지 규칙을 강제하는 것을 말한다(i.e. input : int only -> output : str only)

계약은 주로 사전/사후조건을 명시하지만 때때로 불변식(함수가 실행되는 동안 일정하게 유지되는 것. 로직 확인용. doc-string에 언급)과 부작용(코드 부작용을 doc-string에 언급)을 기술함.

- 사전조건 : 코드 실행전에 체크/처리해야 하는 것.i.e 데이터 유효성 검사.
    - 함수나 메서드가 정상 동작하기 위해 보장해야 하는 조건을 의미함. (i.e. 초기화된 객체,      null 아닌 값등) —> Type checking 보다는 유효성 검사에 가까움.
    - 계약에 의한 디자인(DbC)에서는 함수가 자체적으로 로직을 실행하기전에 유효성 검사를 하는것      이 널리 쓰임.(demanding approach)
    - 위에 반대되는 것은 "클라이언트가 함수를 호출하기 전에 모든 유효성 검사를 하는 것"임(tolerant approach)


- 사후조건(postcondition) : 한수 return값의 유효성 검사, 호출자가 기대하는 바를. 
      return 받았는지 검사.

- 불변식(invariant) :  때로는 함수의 docstring에 불변식에 대해 문서화하는 것이 좋다. 불변식은 함수가 실행되는 동안에 일정하게 유지되는 것으로 함수의 로직에 문제가 없는지 확인하기 위한 것이다

- 부작용(side-effect) : 선택저긍로 코드이 부작용을 docstring에 언급하기도 한다.



##### 계약에 의한 디자인(DbC) - 결론
 - 문제가 있는 부분을 효과적으로 식별할 수 있다.
 - 코드가 더욱 견고해진다.
 - 프로그램 구조 명확히 하는데 도움
 - 계약 작성에 대한 추가작업이 발생하며 단위 테스트를 상황에 따라 추가해야 할 수도 있다.
 - Mypy와 같은 도구를 사용하면 쉽게 할 수 있음

#### 방어적 프로그래밍(Defensive programming)

(Dbc는 계약에서 예외를 발생시키고 실패하게 되는 모든 조건을 기술하는 대신,)
방어적 프로그래밍 프로그래밍은 객체, 함수 또는 메서드와 같은 코드의 모든 부분을 유효하지 않은 것으로부터 스스로 보호할 수 있게 하는 것이다


##### Error handling

- 예상할 수 있는 상황의 오류를 처리하는 방법
- 주 목적 : 예상되는 에러에 대해서 실행이 계속 가능한지 도는 불가능하여 프로그램을 중단할 것인지 결정

- 적용 상황 : 일반적으로 데이터 입력 확인 시 자주 사용됨
- 에러 처리방법
    - 값 대체(substitution)  
    기본값 또는 잘 알려진 상수, 초기값으로 결측값/결과값을 대체.  
     단, 오류 자체를 숨길 수 있으ㅁ로 신중해야함
    - 에외 처리
    예외적인 상황을 알려주고 원래의 비즈니스 로직 흐름을 유지.  
     단, 예외처리를 비즈니스 로직의 일부로 사용하지 말고 호출자가 알아야만하는  
      실질적인 문제 발생에 대해서 사용해야함

- 올바른 수준의 추상화 단계에서 예외 처리
예외는 오직 한가지 일을 한느 함수의 한 부분이어야 한다. 함수가 처리하는( 또는 발생시키는)  
예외는 캡슐화된 로직과 일치해야 한다.


In [2]:
"""Clean Code in Python - Chapter 3: General Traits of Good Code

> Error Handling - Exceptions
"""
import logging
import time

logger = logging.getLogger(__name__)


class Connector:
    """Abstract the connection to a database."""

    def connect(self):
        """Connect to a data source."""
        return self

    @staticmethod
    def send(data):
        return data


class Event:
    def __init__(self, payload):
        self._payload = payload

    def decode(self):
        return f"decoded {self._payload}"


class DataTransport:
    """An example of an object badly handling exceptions of different levels."""

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("connection error detected: %s", e)
            raise
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

    def connect(self):
        for _ in range(self.retry_n_times):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                    "%s: attempting new connection in %is",
                    e,
                    self.retry_threshold,
                )
                time.sleep(self.retry_threshold)
            else:
                return self.connection
        raise ConnectionError(
            f"Couldn't connect after {self.retry_n_times} times"
        )

    def send(self, data):
        return self.connection.send(data)
    
    
"""
ValueError와 ConnectionError -> ConnectionError는 메서드 내에서 처리되어야한다.
"""

In [ ]:
"""Clean Code in Python - Chapter 3: General Traits of Good Code

> Error Handling - Exceptions
"""
import logging
import time

logger = logging.getLogger(__name__)


class Connector:
    """Abstract the connection to a database."""

    def connect(self):
        """Connect to a data source."""
        return self

    @staticmethod
    def send(data):
        return data


class Event:
    def __init__(self, payload):
        self._payload = payload

    def decode(self):
        return f"decoded {self._payload}"


def connect_with_retry(connector, retry_n_times, retry_threshold=5):
    """Tries to establish the connection of <connector> retrying
    <retry_n_times>.

    If it can connect, returns the connection object.
    If it's not possible after the retries, raises ConnectionError

    :param connector:           An object with a `.connect()` method.
    :param retry_n_times int:   The number of times to try to call
                                ``connector.connect()``.
    :param retry_threshold int: The time lapse between retry calls.

    """
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: attempting new connection in %is", e, retry_threshold
            )
            time.sleep(retry_threshold)
    exc = ConnectionError(f"Couldn't connect after {retry_n_times} times")
    logger.exception(exc)
    raise exc


class DataTransport:
    """An example of an object that separates the exception handling by
    abstraction levels.
    """

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        self.connection = connect_with_retry(
            self._connector, self.retry_n_times, self.retry_threshold
        )
        self.send(event)

    def send(self, event):
        try:
            return self.connection.send(event.decode())
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise


In [ ]:
"""Clean Code in Python - Chapter 3: General Traits of Good Code

> Exceptions
"""


class InternalDataError(Exception):
    """An exception with the data of our domain problem."""


def process(data_dictionary, record_id):
    try:
        return data_dictionary[record_id]
    except KeyError as e:
        raise InternalDataError("Record not present") from e

- Traceback 노출금지 : 파이썬의 traceback도 많은 디버깅 정보를 포함하기 때문에  
    역으로 악의적인 사용자에게도 유용한 정보.  
  예외가 전파되도록 할 때, 중요한 정보를 공개하지 않도록 주의필요
  
- 비어있는 except 블록 지양  
에러에 대한 지나치게 방어적인 프로그래밍은 더 큰 문제를 야기한다.

    - 보다 구체적인 예외 사용(Exception 같이 광범위한 예외를 사용하면 안된다.  
    실제로 코드가 너무 광범위한 예외를 사용하는 경우 린팅 도구나 IDE에서 경고를 표시한다
    
    -except 블록에서 실제 오류 처리를 한다.
    
    좋은 방법은 두 항목을 동시에 적용하는 것

- 원분 에외 포함 : 오류 처리 과정에 다른 오류를 발생시키고 메시지를 변경 가능  
                이 경우 원래 예외를 포함하는 것이 좋다  
  파이썬3(PEP-3134)에서는 raise <e> from <original_exception> 구문 사용  
  원본의 tracebackdl 새로운 exception에 포함되고 원본 예외는 __cause__ 속석으로 설정

In [ ]:
class InterruptedError(Exception):
    """업무 도메인 데이터의 예외"""

def process(data_dictionary, record_id):
    try:
        return data_dictionary[record_id]
    except KeyError as e:
        raise InternalDataError("Record not present") from e

"""Tip: 예외의 타입을 변경할 때 항상 raise <e> from <o> 구문을 사용한다"""

#### 파이썬에서 assertion 사용하기

- 절대로 일어나지 않아야 하는 상황에 사용되므로 assert 문에 사용된 표현식은 불가능한 조건을 의미  
이 상태가 된다는건 소프트웨어에 결함이 있음을 의미

- 에러 발생시 바로 프로그램이 중단.  
그렇기 때문에 assertion을 비즈니스 로직과 섞거나 소프트웨어 제어 흐름 메커니즘으로 사용 x


assertion 사용의 안좋은 예

In [ ]:
try:
    assert condition.holds(),   "조건에 맞지 않음."
except AssertionError:
    alternative_procedure()

"""AssertionError 예외를 처리하지 않는다"""

개선 ->

In [ ]:
result = condition.holds()
assert result > 0, "에러 {0}".format(result)

#### 관심사의 분리(Separation of concerns)

여기서 말하는 관


##### 응집력
: 객체가 작고 잘 정의된 목적을 가져야 함을 의미. 응집력이 높을수록 좋음


##### 결합력
: 두개 이상의 객체간의 의존성을 나타내며 서로간의 의존성/종속성이 클수록 바람직하지 않음  

    - 낮은 재사용성 : 다른 상황에서 이 함수를 사용하기가 매우 어려움.  
    - 파급효과 : 한 군데의 수정이 다른 부분에도 영향을 크게 미침.  
    - 낮은 수준의 추상화 : 밀접한 두 함수가 서로 다른 추상화 레벨에서 문제를 해결하기 어려움.  
        ->관심사가 분리되어 있다고 보기 어려움

+ 높은 응집력과 낮은 결합력을 갖는 것이 좋은 소프트웨어이다.

#### 개발 지침 약어(Acronyms to live by)

좋은 소프트웨어 관행을 약어로 통해 쉽게 소통

#### DRY/OAOO
: Do not repeat yourself, Once and only once를 의미하며 코드에 있는 지식이 단 한번, 단 한곳에 정의되어야 하고 또한 코드를 변경하려고 할때 수정이 필요한 곳이 한군데에만 있어야 한다는 SW best practice. 코드 중복은 아래와 같은 리스트를 발생시킴

- 오류가 발생하기 쉽다 : 여러번 반복된 코드가 있다면 수정과정에서 오류 발생 가능성이 높아진다.
- 비용이 비싸다. : 변경시 더 많은 시간이 들며, 팀 전체의 개발 속도를 지연시킨다.
- 신뢰성이 떨어진다. : 문맥상 여러 코드를 변경하는 경우, 사람이 모든 인스턴스의 위치를 기억해야 하며, 단일 데이터 소스가 아니므로 데이터 완결성도 저하된다.

Author) 중복을 제거하는 방법은 간단하게는 함수 생성 기법부터 컨텍스트 관리자를 이용하거나 이터레이터, 제너레이터, 데코레이더등을 활용할 수도 있다

##### YANGI
- You ain't gonna need it을 의미하며 과잉 엔지니어링(굳이 필요없는 개발)을 자제하라는 의미이다. 유지보수가 가능한 소프트웨어는 미래 욕두 사항을 예측하는것이 아닌 오직 현재의 요구상항ㅇ르 잘해결하기 위한 소프트웨어르 수정하기 쉽게 작성하는 것이다


#### KIS
- Keep it simple을 의미하며 YANGI와 유사. 문제를 올바르게 해결하는 최소한의 기능을 구현하고 필요 이상으로 솔루션을 복잡하게 만들지 않는다. 단순하게 유지하여 관리르 용이하게 한다.

In [ ]:
class ComplicateNamespace:
    """프로피터를 가진 객체를 초기화하는 복잡한 예제"""
    
    ACCEPED_VALUES = {"id_", "user", "location"}
    
    @classmethod
    def init_with_data(cls, **data):
        isinstance = cls()
        for key, value in data.items():
            setattr(instance, key, value)
        
        return isinstance

In [ ]:
class Namespace:
    """Create an Object from keyword arguments."""

#### EAFP/LBYL

- Easier to ask forgiveness than permission, look before you leap를 의미한다. 전자는 일단 코드를 실행하고 실제 동작하지 않을 경우에 대항한다는 뜻이며, 후자는 사전에 미리 확인하고 실행하라는 의미이다.

In [ ]:
import os

if os.path.exists(filename):
    with open(filename) as f:
        ...
        
    try:
        with open(filename) as f:
            ...
    except FileNotFoundError as e:
        logger.error(e)
        

#### 컴포지션과 상속
- OOP의 핵심개념인 상속의 장점과 단점을 알아보고 적절한 방법을 논한다. 코드 상속을 통한 코드 제사용에 있어서 올바른 방법은, 여러 상황에서 동작 간으하고 쉽게 조합할 수 있는 응집력 높은 객체를 개발/ 사용하는  것이다.(동시에 결합력이 낮은)


#### 상속이 좋은 선택인 경우
- 파생 클래스 양날의 검. 부모 클래스의 메서드를 쉽게 전수 받을 수 있음 but 너무 많으 기능을 가질 수 있어 사용장의 혼란을 가중시킬 수 있다는 단점 존재

- 상속의 적용이 적절하지 않은 경우
    - 부모 클래스가 막연한 정의와 너무 많은 책임을 가짐
    - 자식 클래스는 부모 클래스의 적절한 세분화가 아님
- 상속이 좋은 선택인 경우(일반적 클래스를 전문화/세부 추상화)
    - 부모 클래스의 기능을 모두 필요 + 추가 기능 또는 특정기능 추가
    http.server패키지의 BaseHTTPRequestHandler 기본 클래스 일부를 추가하거나  
    변경하여 확장하는 SimpleHTTPRequestHandler 하위 클래스. 
    (https://docs.python.org/3/library/http.server.html#http.serverBaseHITTPRequestHandler)
    
    - 객체의 인터페이스 방식을 강제하고자 할때 구현 하지 않은 추상적인 부모 클래스를 만들고,  
     이 클래스를 상속하는 자식 클래스에서 적절한 구현을 하는 경우.
     
    - 파이썬의 표준 예외는 Exception에서 파생. 
    ex) HTTPError는 RequestException를 상속받고, RequestException은 IOError 상속


##### 상속 안티패턴

- 도메인 문제를 해결하기 위해 적절한 데이터 구조를 만든 다음에 이 데이터 구조를 사용하는 객체를 만들지 않고 데이터 구조 자체를 객체로 만드는 경우

In [ ]:
class TransactionalPolicy(collections.UserDict):
    """잘못된 상속의 예"""

    def change_in_policy(self, customer_id, **new_policy_data):
        self[customer_id].update(**new_policy_data)

>>> policy = TransactionalPolicy({
					"client001":{
							"fee":1000.0, 
							"expiration_date":datetime(2020,1,3),
					}
		})
>>> policy["client001"]
{"fee":1000.0, "expiration_date":datetime.datetime(2020,1,3,0,0)}

>>> policy.change_in_policy("client001", expiration_date=datetime(2020,1,4))
>>> policy["client001"]
{"fee":1000.0, "expiration_date":datetime.datetime(2020,1,4,0,0)}

>>>dir(policy)
[# 모든 매직 메서드는 생략...
	'change_in_policy','clear','copy','data','fromkeys','get','items','keys','pop',
	'popitem','setdefault','update','values']

- 위의 코드에서 문제점
    - TransactionalPolicy 이름을 통해 사전 타입이라는 것을 파악하기 어렵고, 노출된 public 메서드를 통해
    부적절하게 전문화된 이상한 계층 구조라고 사용자들이 생각 할 것임.
    
    - 불필요해보이는 pop, items와 같은 public 메서드들이 그대로 노출되어 있어서 사용자들이 임의로 호출할 수 있음
    
- 컴포지션을 활용한 개선

: dictionary를 private 속성에 저장하고 __getitem__()으로 사전의 프록시를 만들고 나머지 필요한 메서드를 추가적으로 구현


In [ ]:
class TransactionalPolicy:
    """컴포지션을 사용한 리팩토링"""

    def __init__(self, policy_data, **extra_data):
        self._data = {**policy_data, **extra_data}

    def change_in_policy(self, customer_id, **new_policy_data):
        self._data[customer_id].update(**new_policy_data)

    def __getitem__(self, customer_id):
        return self._data[customer_id]

    def __len__(self):
        return len(self._data)

#### 파이썬의 다중상속
다중 상속을 지원하는 파이썬은 자칫 잘못 구현하면 여러가지 디자인 문제를 유발

- 메서드 결정 순서(MRO)

: 다중 상속 구조 다이어그램

![nn](MRO_diagram.png)

- 최상위 클래스는 moudule_name 속성을 가지며 __str__ 메서드 구현
BaseModule1,2 하단 클래스 ConcreteModuleA12 ,B23    확인


In [2]:
"""Clean Code in Python - Chapter 3: General traits of good code

> Multiple inheritance: MRO

"""

class BaseModule:
    module_name = "top"

    def __init__(self, module_name):
        self.name = module_name

    def __str__(self):
        return f"{self.module_name}:{self.name}"


class BaseModule1(BaseModule):
    module_name = "module-1"


class BaseModule2(BaseModule):
    module_name = "module-2"


class BaseModule3(BaseModule):
    module_name = "module-3"


class ConcreteModuleA12(BaseModule1, BaseModule2):
    """Extend 1 & 2

    >>> str(ConcreteModuleA12('name'))
    'module-1:name'
    """


class ConcreteModuleB23(BaseModule2, BaseModule3):
    """Extend 2 & 3

    >>> str(ConcreteModuleB23("test"))
    'module-2:test'
    """

In [3]:
str(ConcreteModuleA12('test'))

"""충돌 발생 x  파이썬은 C3 linearization 또는 MRO 라는 알고리즘으로 이문제 해결"""

'module-1:test'

In [4]:
#구체적으로 클래스에게 결정 순서를 직접 물어볼 수도 있다.

[cls.__name__ for cls in ConcreteModuleA12.mro()]

#메서드가 계층 구조에서 어떻게 해결되는 지 알고 있다면 mixin을 이해나는데 도음이 많이 됨


['ConcreteModuleA12', 'BaseModule1', 'BaseModule2', 'BaseModule', 'object']

#### mixin(믹스인)
: 코드를 재사용하기 위해 일반적인 행동을 캡슐화해놓은 기본 클래스

- 믹스인 클래스는 그 자체로 유용하지 않으며 대부분 클래스에 정의된 매서드나 속성에 의존

- 다른 클래스와 함께 믹스인 클래스를 다중 상속하여 믹스인 내부 메서드/속성을 사용

In [5]:
"""문자열을 받아서 하이픈(-)으로 구분된 값을 반환하는 파서"""

class BaseTokenizer:
    def __init__(self, str_token):
        self.str_token = str_token
        
    def __iter__(self):
        yield from self.str_token.split("-")
        
        

In [6]:
tk = BaseTokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0") 
list(tk)



['28a2320b', 'fd3f', '4627', '9792', 'a2b38e3c46b0']

In [7]:
"""기본 클래스를 변경하지 않고 값을 대문자로 변환해보자"""

class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())
    
class Tokenizer(UpperIterableMixin, BaseTokenizer):
    pass

##### Arguments in functions and methods(함수와 메서드의 인자)
: 함수의 인자 전달 메커니즘과 SW 개발 모범사례에서 발견되는 일반적인 원칙을 살펴본다.

##### 파이썬의 함수 인자 동작방식
- 파이썬에서는 모든 인자가 값에 의해 전달된다. 즉 함수에 값을 전달하면 함수의 변수에 할당되고 나중에 사용된다

- 만약 변형가능한(mutable) 객체를 전달하고 함수 내부에서 값을 변경하면 결과에서 실제 값이 변경될 수 있다

In [8]:
def function(arg):
    arg += " in function"
    print(arg)
    
immutable = "hello"
function(immutable)

hello in function


In [11]:
mutable = list("hello")
print(immutable)
function(mutable)

hello
['h', 'e', 'l', 'l', 'o', ' ', 'i', 'n', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n']


Tip: 함수 인자를 변경하지 않아야 한다. 최대한 함수에서 발생할 수 있는 부작용을 회피하라

##### 가변인자
- 해당 인자를 패킹(packing)할 변수의 이름 앞에 별표(*)를 사용 

In [2]:
def f(first, second, thrid):
    print(first)
    print(second)
    print(thrid)
    
l = [1, 2, 3]
f(*l)

1
2
3


In [3]:
a, b, c = [1, 2, 3]
print(a)
print(b)
print(c)

1
2
3


In [9]:
def show(e, rest):
    print("요소: {0} - 나머지 : {1}".format(e, rest))

first, *rest = [1,2, 3, 4, 5]
show(first, rest)

*rest, last = range(6)
show(last, rest)

first, *middle, last = range(6)
print(first)
print(middle)
print(last)

요소: 1 - 나머지 : [2, 3, 4, 5]
요소: 5 - 나머지 : [0, 1, 2, 3, 4]
0
[1, 2, 3, 4]
5


In [11]:
first, last, *empty = (1,2)
print(first)
print(last)
print(empty)

1
2
[]


변수 언패킹의 가장 좋은 사용 예는 반복이다. 일련의 요소를 반복해야 하고 각 요소가 차례로 있다면 각 요소를 반복할 때 언패킹하는 것이 좋다

In [1]:
USERS = [(i, f"fisrt_name_{i}", "last_name_{i}") for i in range(1_000)]

class User:
    def __init__ (self, user_id, first_name, last_name):
        self.user_id = user_id
        self.first_name = first_name
        self.last_name = last_name
    
    def bad_users_from_rows(dbrows)->list:
        """db_record에서 사용자를 생성하는 파이썬스럽지 않은 잘못된 사용 예"""
        return [User(row[0], row[1], row[2]) for row in dbrows]
    
    def users_from_rows(dbrows) -> list:
        """db 레코드에서 사용자 생성"""
        return [User(user_id, first_name, last_name)
                for(user_id, first_name, last_name) in dbrows
                ]


#### 함수 인자의 개수
: 함수가 너무 많은 인자를 가진다면, 나쁜 디자인일 가능성이 높다.

1. 구체화(전달하는 모든 인자를 포함하는 새로운 객체를 만듦)를 통해 일반적 sw 디자인의 원칙을 사용

2. 가변인자나 키워드 인자를 사용하여 동적 서명으로 가진 함수를 만듬. 단, 매우 동적이어셔 유지 보수가 어렵기

3. 여러 작은(한가지 기능을 담당하는) 함수로 분리한다.

- 함수 인자와 결합력
    - 함수의 인자가 많을수록 호출하기 위한 정보를 수집하기가 점점 어려워진다.
    
    - 함수의 추상화가 너무 적으면 다른 환경에서 사용하기 어렵거나(재사용성x), 다른 함수에 의존적
    
    - 함수가 너무 많은 인자를 가진다면, 나쁜 디자인을 가능성이 높다
    
- 많은 인자를 취하는 작은 함수의 서명
    - 공통 객체에 파라미터 대부분이 포함되어 있는 경우 리팩토링 하기
    track_request(request.headers, request.ip_addr, request.request_id)

#### 소프트웨어 독립성(Orthogonality)
:SW의 런타임 구조 측면에서 직교란, 변경(또는 부작용)을 내부 문제로 만드는 것이다.  
예를들어 어떤 객체의 메서드를 호출하는 것이 다른 관련없는 객체의 내부 상태를 변경 하면 안된다.

- 가격 계산함수와 표시함수는 서로 독립

In [5]:
def calculate_price(base_price: float, tax:float, discount: float) -> float:
    return (base_price * (1+tax)) *(1 - discount)

def show_price(price: float) ->str:
    return "$ {0:, .2f}".format(price)

def str_final_price(
    base_price : float, tax: float, discount: float, fmt_function= str)->str:
    return fmt_function(calculate_price(base_price), tax, discount)

In [ ]:
str_final_price(10, 0.2, 0.5)

#### 코드 구조
: 팀의 작업 효울성과 유지보수성을 위해 코드 구조화

- 유사한 컴포넌트끼리 정리하여 구조화

- init.py를 가진 디렉토리를 통해 파이썬 패키지를 만들면, 코드간의 종속성이 있어도 전체적인  
호환성을 유지 (init.py 파일에 다른 파일에 있던 모든 정의를 가져올 수 있음)
    - 모듈을 import할때 구문을 분석하고 메모리에 로드할 객체가 줄어든다
    - 의성이 줗었기 때문에 더 작은 모듈만 가져오면 된다.

- config에 대한 정의가 필요한 사우를 정의하고 일괄 import하여 정보를 중앙화